In [28]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

import os
from dotenv import load_dotenv

In [16]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


In [29]:
load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 3000,
    GenParams.MIN_NEW_TOKENS: 1,
    # GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1,
    GenParams.STOP_SEQUENCES: ['```']
}

model = Model("meta-llama/llama-2-70b-chat", creds, params, project_id)

In [24]:
from datetime import datetime, timedelta

def build_nvd_url(days):
    base_url = "https://services.nvd.nist.gov/rest/json/cves/2.0/"

    # Calculate the start and end dates
    end_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000")
    start_date = (datetime.now() - timedelta(days=days)).strftime("%Y-%m-%dT%H:%M:%S.000")

    # Build the URL string
    url = f"{base_url}?pubStartDate={start_date}&pubEndDate={end_date}"
    return url

In [33]:
import requests

response = requests.get(build_nvd_url(3))
response.raise_for_status()
# print(response.text())

# Extract the vulnerability information from the API response
vulnerability_data = response.json()
# print(vulnerability_data)
print(len(vulnerability_data['vulnerabilities']))
for vulnerability in vulnerability_data['vulnerabilities']:
    # Extract the relevant details from the vulnerability data
    cve_id = vulnerability['cve']['id']
    description = vulnerability['cve']['descriptions'][0]['value']
    published_date = vulnerability['cve']['published']

    # Print the vulnerability details
    print(f"Latest Vulnerability:")
    print(f"CVE ID: {cve_id}")
    print(f"Description: {description}")
    print(f"Published Date: {published_date}")

    prompt = f"""[INST]you be security expert. you read into the vulnerability data in backquoted, and generate summary within 300 words:
    cve:`{vulnerability}`
    [/INST]summary:"""

    summary = ""
    for response in model.generate_text(prompt):
        summary += response

    file_path = os.path.join("cve", f"{cve_id}.txt")

    # Save the vulnerability data to the JSON file
    with open(file_path, 'w') as file:
        file.write(summary)

    # print(summary)

90
Latest Vulnerability:
CVE ID: CVE-2023-50951
Description: IBM QRadar Suite 1.10.12.0 through 1.10.17.0 and IBM Cloud Pak for Security 1.10.0.0 through 1.10.11.0 in some circumstances will log some sensitive information about invalid authorization attempts.  IBM X-Force ID:  275747.
Published Date: 2024-02-17T16:15:46.443
Latest Vulnerability:
CVE ID: CVE-2024-22335
Description: IBM QRadar Suite 1.10.12.0 through 1.10.17.0 and IBM Cloud Pak for Security 1.10.0.0 through 1.10.11.0 stores potentially sensitive information in log files that could be read by a local user.  IBM X-Force ID:  279975.
Published Date: 2024-02-17T16:15:47.000
Latest Vulnerability:
CVE ID: CVE-2024-22336
Description: IBM QRadar Suite 1.10.12.0 through 1.10.17.0 and IBM Cloud Pak for Security 1.10.0.0 through 1.10.11.0 stores potentially sensitive information in log files that could be read by a local user.  IBM X-Force ID:  279976.
Published Date: 2024-02-17T16:15:47.190
Latest Vulnerability:
CVE ID: CVE-2024-2